# Ion Exchange Simulation - Na-WAC → Degasser

This notebook performs detailed simulation of the simple Na-WAC flowsheet.

## Process Description
1. **Na-WAC**: Weak Acid Cation resin in sodium form removes temporary hardness
2. **Degasser**: Strips CO2 formed from bicarbonate conversion

Best for: Simple water chemistry with moderate hardness and limited permanent hardness

⚠️ **Note**: This flowsheet CANNOT remove permanent hardness!

In [ ]:
# Parameters cell - papermill will inject values here
import json

# Configuration parameters
configuration = None  # Will be injected by papermill
water_analysis = None  # Will be injected by papermill
simulation_options = {
    "model_type": "direct",
    "time_steps": 100,
    "breakthrough_criteria": {"hardness_mg_L_CaCO3": 5.0}
}

In [ ]:
# Import required modules
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Add project root to path
if configuration and 'project_root' in configuration:
    sys.path.insert(0, configuration['project_root'])

from tools.ix_simulation_direct import simulate_ix_system_direct
from tools.schemas import IXSimulationInput, IXConfigurationOutput, MCASWaterComposition

## Water Suitability Check

In [ ]:
# Check if water is suitable for Na-WAC treatment
if water_analysis:
    ca = water_analysis['ion_concentrations_mg_L'].get('Ca_2+', 0)
    mg = water_analysis['ion_concentrations_mg_L'].get('Mg_2+', 0)
    hco3 = water_analysis['ion_concentrations_mg_L'].get('HCO3_-', 0)
    
    total_hardness = ca * 2.5 + mg * 4.1
    alkalinity = hco3 / 1.22
    temp_hardness = min(total_hardness, alkalinity)
    perm_hardness = max(0, total_hardness - alkalinity)
    
    print("Water Suitability Analysis for Na-WAC:")
    print(f"Total Hardness: {total_hardness:.1f} mg/L as CaCO3")
    print(f"Temporary Hardness: {temp_hardness:.1f} mg/L ({temp_hardness/total_hardness*100:.0f}%)")
    print(f"Permanent Hardness: {perm_hardness:.1f} mg/L ({perm_hardness/total_hardness*100:.0f}%)")
    
    if perm_hardness > 50:
        print("\n⚠️ WARNING: High permanent hardness ({perm_hardness:.0f} mg/L)!")
        print("Na-WAC cannot remove permanent hardness. Consider SAC or H-WAC flowsheet instead.")
    elif perm_hardness > 20:
        print("\n⚠️ CAUTION: Moderate permanent hardness will pass through untreated.")
    else:
        print("\n✓ Water is suitable for Na-WAC treatment.")

## Run Simulation

In [ ]:
# Prepare simulation input
if configuration and water_analysis:
    # Create proper schema objects
    config_obj = IXConfigurationOutput(**configuration)
    water_obj = MCASWaterComposition(**water_analysis)
    
    sim_input = IXSimulationInput(
        configuration=config_obj,
        water_analysis=water_obj,
        breakthrough_criteria=simulation_options['breakthrough_criteria'],
        simulation_options=simulation_options
    )
    
    print("Running Na-WAC flowsheet simulation...")
    result = simulate_ix_system_direct(sim_input)
    print(f"Simulation status: {result.status}")

## Alkalinity Removal

In [ ]:
# Show alkalinity removal through the system
if result and result.water_quality_progression:
    stages = []
    alkalinity = []
    hardness = []
    ph = []
    
    for prog in result.water_quality_progression:
        stages.append(prog.stage)
        alkalinity.append(prog.alkalinity_mg_L_CaCO3)
        hardness.append(prog.hardness_mg_L_CaCO3)
        ph.append(prog.pH)
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
    
    # Alkalinity and hardness
    x = range(len(stages))
    ax1.bar(x, alkalinity, width=0.4, label='Alkalinity', align='edge')
    ax1.bar([i+0.4 for i in x], hardness, width=0.4, label='Total Hardness', align='edge')
    ax1.set_ylabel('Concentration (mg/L as CaCO3)')
    ax1.set_title('Alkalinity and Hardness Removal')
    ax1.legend()
    ax1.grid(True, axis='y')
    
    # pH changes
    ax2.plot(x, ph, 'ro-', linewidth=2, markersize=8)
    ax2.set_ylabel('pH')
    ax2.set_xlabel('Treatment Stage')
    ax2.set_xticks(x)
    ax2.set_xticklabels(stages, rotation=45, ha='right')
    ax2.grid(True)
    ax2.set_ylim(4, 9)
    
    plt.tight_layout()
    plt.show()
    
    # Show removal efficiency
    feed_alk = alkalinity[0]
    final_alk = alkalinity[-1]
    alk_removal = (feed_alk - final_alk) / feed_alk * 100
    
    print(f"\nAlkalinity Removal: {alk_removal:.1f}%")
    print(f"Feed: {feed_alk:.0f} mg/L → Product: {final_alk:.0f} mg/L")

## Simplicity Advantages

In [ ]:
# Compare Na-WAC advantages vs other flowsheets
if result and result.economics:
    print("Na-WAC System Advantages:")
    print("\n1. Simplicity:")
    print("   - Only one IX stage (fewer vessels)")
    print("   - Single regenerant chemical (NaCl)")
    print("   - Lower operator skill requirements")
    
    print("\n2. Economics:")
    print(f"   - Capital Cost: ${result.economics.get('capital_cost_usd', 0):,}")
    print(f"   - Annual OPEX: ${result.economics.get('annual_opex_usd', 0):,}")
    print(f"   - Cost per m³: ${result.economics.get('cost_per_m3', 0):.2f}")
    
    print("\n3. Operation:")
    print("   - High capacity for temporary hardness")
    print("   - Neutral pH effluent (after degassing)")
    print("   - Less waste volume than SAC systems")
    
    print("\n4. Limitations:")
    print("   - Cannot remove permanent hardness")
    print("   - Not suitable for all water types")
    print("   - May need polishing for strict limits")

## Regeneration Profile

In [ ]:
# Show regeneration requirements
if result and result.ix_performance:
    nawac_perf = result.ix_performance.get('Na-WAC')
    if nawac_perf:
        print("Na-WAC Regeneration Profile:")
        print(f"\nChemical: {nawac_perf.regenerant_chemical}")
        print(f"Consumption: {nawac_perf.regenerant_consumption_kg:.1f} kg/cycle")
        print(f"Specific usage: {nawac_perf.specific_regenerant_g_L:.0f} g/L resin")
        print(f"Concentration: 10% brine solution typical")
        
        # Calculate brine volume
        brine_volume_m3 = nawac_perf.regenerant_consumption_kg / 100  # 10% solution
        print(f"\nBrine volume: {brine_volume_m3:.1f} m³/cycle")
        print(f"Rinse volume: {nawac_perf.waste_volume_m3:.1f} m³/cycle")
        print(f"Total waste: {brine_volume_m3 + nawac_perf.waste_volume_m3:.1f} m³/cycle")
        
        # Annual projections
        cycles_per_year = 8760 / nawac_perf.breakthrough_time_hours
        annual_salt = nawac_perf.regenerant_consumption_kg * cycles_per_year / 1000
        annual_waste = (brine_volume_m3 + nawac_perf.waste_volume_m3) * cycles_per_year
        
        print(f"\nAnnual Projections:")
        print(f"Regeneration cycles: {cycles_per_year:.0f}/year")
        print(f"Salt consumption: {annual_salt:.1f} tons/year")
        print(f"Waste generation: {annual_waste:.0f} m³/year")

## Operational Guidelines

In [ ]:
# Na-WAC specific operational recommendations
print("Na-WAC System Operational Guidelines:")
print("\n1. Service Cycle:")
print("   - Monitor alkalinity breakthrough (not just hardness)")
print("   - pH will drop as alkalinity breaks through")
print("   - Terminate at pH 6.5 or hardness >5 mg/L")

print("\n2. Regeneration Best Practices:")
print("   - Use high-quality salt (>99.5% NaCl)")
print("   - 10% brine concentration optimal")
print("   - Slow flow rate: 2-4 BV/hr")
print("   - Rinse until conductivity <500 µS/cm")

print("\n3. Troubleshooting:")
print("   - Early breakthrough → Check for channeling")
print("   - High leakage → Verify regeneration level")
print("   - pH issues → Check degasser operation")

print("\n4. Maintenance:")
print("   - Backwash weekly or as needed")
print("   - Check resin annually for fines")
print("   - Clean degasser packing quarterly")

if result and result.warnings:
    print("\n5. System-Specific Warnings:")
    for warning in result.warnings:
        print(f"   ⚠️ {warning}")

In [ ]:
# Prepare results for extraction
results = result.dict() if result else {"status": "error", "message": "No simulation results"}
results